In [35]:
import pandas as pd
import numpy as np
import torchvision
import torch
import random
import os 

from tqdm import tqdm
from pathlib import Path
from mouse_facial_expressions.data.datasets import Task1Folds
from mouse_facial_expressions.data.make_datasets import get_treatment_video_dataframe
from torch.utils.data import DataLoader

In [2]:
os.environ['MFE_EXTRACTED_FRAMES_FOLDER']

'/home/andre/calgary2023/extracted_frames'

In [2]:
seed = 42
cv = Task1Folds()
for train_dataset, test_dataset in cv:
    random.seed(seed)
    np.random.seed(seed)
    torch.use_deterministic_algorithms(True)
    torch.manual_seed(seed)
    # For seeding dataloaders
    g = torch.Generator()
    g.manual_seed(seed)
    
    train_dataloader = DataLoader(train_dataset, batch_size=20, num_workers=8, shuffle=True, generator=g)
    test_dataloader = DataLoader(test_dataset, batch_size=20)

In [6]:
temp = train_dataset.get_eval_mask()
temp

recording,0,1,2,3,4
mouse,,,,,
f1,1,0,1,1,1
f10,1,0,1,1,1
f11,1,0,1,1,1
f12,1,0,1,1,0
f13,1,0,1,1,1
f14,1,0,1,1,1
f15,1,0,1,1,0
f16,1,0,1,1,1
f2,1,0,1,1,1


In [39]:
import mlflow
import matplotlib.pyplot as plt 
import pandas as pd

client = mlflow.MlflowClient('../mlruns')

experiment = client.search_experiments()[0]
experiment_id = experiment.experiment_id
print(experiment.name)

experiment_runs = client.search_runs(
    experiment_id, 
    # "tags.mlflow.source.name='file:///workspaces/telfer-ecg-heartbeat-categorization-task'",
    order_by=["start_time"])


# last_parent_run = parent_runs[-1]

baseline


In [40]:
artifacts = [f.path for run in experiment_runs for f in client.list_artifacts(run.info.run_id)]
artifacts

[]

In [19]:
for run in experiment_runs:
    run

<Run: data=<RunData: metrics={}, params={'epochs': '3', 'seed': '42'}, tags={'mlflow.gitRepoURL': 'git@github.com:A-Telfer/mouse-facial-expressions-2023.git',
 'mlflow.project.entryPoint': 'train',
 'mlflow.project.env': 'conda',
 'mlflow.runName': 'crawling-skunk-346',
 'mlflow.source.git.commit': '037736f47f4f089c30e6d9cf8a94b761c463f814',
 'mlflow.source.git.repoURL': 'git@github.com:A-Telfer/mouse-facial-expressions-2023.git',
 'mlflow.source.name': 'file:///home/andre/repos/mouse-facial-expressions-2023',
 'mlflow.source.type': 'PROJECT',
 'mlflow.user': 'andre'}>, info=<RunInfo: artifact_uri='file:///home/andre/repos/mouse-facial-expressions-2023/mlruns/523496863183153064/08fafc7ea4064808b0def7a647bef33c/artifacts', end_time=None, experiment_id='523496863183153064', lifecycle_stage='active', run_id='08fafc7ea4064808b0def7a647bef33c', run_name='crawling-skunk-346', run_uuid='08fafc7ea4064808b0def7a647bef33c', start_time=1689313841888, status='RUNNING', user_id='andre'>, inputs=<Ru

In [7]:
temp.columns

MultiIndex([('can_test', 0),
            ('can_test', 1),
            ('can_test', 2),
            ('can_test', 3),
            ('can_test', 4)],
           names=[None, 'recording'])

In [25]:
train_samples = {i for s in train_dataset.samples for i in s['indices']}
test_samples = {i for s in test_dataset.samples for i in s['indices']}

train_samples & test_samples

set()

In [4]:
# training_videos = train_dataset.df.loc[list(train_samples)].video.unique()
df = get_treatment_video_dataframe()
df = df[['video', 'mouse', 'treatment', 'recording']]
df = df.drop_duplicates()

training_videos = train_dataset.data_in_samples.video.unique()
temp = df.copy()
temp['can_test'] = 0
temp.loc[~df.video.isin(training_videos), 'can_test'] = 1
temp = temp.pivot(index=['mouse'], columns=['recording'], values=['can_test']).sort_index()
temp.fillna(-1).astype(int)

can_test            
recording        0  1  2  3  4
mouse                         
f1               1  0  1  1  1
f10              1  0  1  1  1
f11              1  0  1  1  1
f12              1  0  1  1  0
f13              1  0  1  1  1
f14              1  0  1  1  1
f15              1  0  1  1  0
f16              1  0  1  1  1
f2               1  0  1  1  1
f20              1  1  1  1  1
f21              1  1  1  1  1
f22              1  1  1  1  1
f23              1  1  1  1  1
f24              1  1  1  1  1
f25              1  1  1  1  1
f26              1  1  1  1  1
f27              1  1  1  1  1
f3               1  1  1  1  1
f30              1  1  1  1  1
f31              1  0  1  1  0
f32              1  0  1  1  1
f33              1  0  1  1  1
f4               1  1  1 -1  1
f5               1  0  1  1  1
f6               1  0  1  1  1
f7               1  0  1  1  1
f8               1  0  1  1  0
f9               1  0  1  1  1
m1               1  0  1  1  1
m10              1  0  1  1  1
m11              1  0  1  1  1
m12              1  0  1  1  1
m13              1  0  1  1  1
m14              1  0  1  1  1
m15              1  0  1  1  0
m16              1  0  1  1  1
m17              1  0  1  1  1
m18              1  1  1  1  1
m19              1  0  1  1  0
m2               1  1  1 -1  1
m20              1  0  1  1  1
m21              1  1  1  1  1
m22              1  1  1  1  1
m23              1  1  1  1  1
m24              1  1  1  1  1
m25              1  1  1  1  1
m26              1  1  1  1  1
m27              1  1  1  1  1
m28              1  1  1  1  1
m29              1  1  1  1  1
m3               1  0  1  1  1
m30              1  1  1  1  1
m4               1  0  1  1  0
m5               1  0  1  1  0
m6               1  0  1  1  1
m7               1  0  1  1  1
m8              -1  0  1  1  1
m9               1  0  1  1  0

In [ ]:
train_dataset.df.loc[list(train_samples)]

In [7]:
import torch

torch.tensor([0,2,3]).reshape(-1, 1).tile((1,3)).reshape(-1)


tensor([0, 0, 0, 2, 2, 2, 3, 3, 3])

In [12]:
for idx, batch in tqdm(enumerate(train_dataloader)):
    break

0it [00:00, ?it/s]

0it [00:00, ?it/s]


In [15]:
x, y = batch
x.shape

torch.Size([20, 5, 3, 300, 300])